# 📊  Enrollment Mismatch Checker
* Compares student rosters between paired main and support CRNs (e.g., MATH 1001 & 0997)
* Flags mismatches where students are enrolled in one but not the other
* Outputs results to an Excel file with preserved IDs, names, emails and timestamps

## 🔹 Load Libraries and Packages

In [78]:
#!pip install XlsxWriter

In [79]:
# 📦 Setup
from google.colab import files, drive
import pandas as pd
import numpy as np
import re
from datetime import datetime


In [80]:
#Convert campus codes to campus names
campus_lookup = {
    'PA': 'Alpharetta',
    'PC': 'Clarkston',
    'PN': 'Dunwoody',
    'PE': 'Newton',
    'PS': 'Decatur',
    'PF': 'Online',
}

In [81]:
# 📄 Load class roster and schedule files
schedule_url = 'https://raw.githubusercontent.com/brownt47/mismatch_support_rosters/main/assets/CSV_Class_Schedule_Summer_Demo.csv'
schedule_df = pd.read_csv(schedule_url)

In [82]:
roster_url = 'https://raw.githubusercontent.com/brownt47/mismatch_support_rosters/main/assets/Class_Rosters_Demo.csv'
roster_df = pd.read_csv(roster_url)


In [83]:
# Convert all column names to lowercase
roster_df.columns = roster_df.columns.str.lower()
schedule_df.columns = schedule_df.columns.str.lower()

In [84]:
# Ensure CRNs are strings for consistency
roster_df['ssbsect_crn'] = roster_df['ssbsect_crn'].astype(str)
schedule_df['crn'] = schedule_df['crn'].astype(str)

In [85]:
# Ensure student IDs are cast as string not to lose leading zeros - 002123456 -> 2123456
roster_df['id'] = roster_df['id'].astype(str).str.zfill(9)  # or adjust length as needed

In [86]:
# Identify courses with support CRNs
schedule_df['has_support'] = (
    schedule_df['subj_crsnumb'].isin(['MATH 0999', 'MATH 0997', 'MATH 0996',
                                      'MATH 1111', 'MATH 1001', 'MATH 1401']) &
    schedule_df['comments'].str.upper().str.contains('CRN', na=False)
).astype(int)

In [87]:
# Create filtered copy of only support linked courses
support_df = schedule_df[schedule_df['has_support'] == 1].copy()

In [88]:
# 🔗 Extract linked CRNs
support_df['linked_crn'] = support_df['comments'].str.extract(r'(?:CRN[:\s#-]*?)?(\d{5})', expand=False)
support_df['linked_crn'] = support_df['linked_crn'].astype(str)

In [89]:
# Build a lookup: CRN → subj_crsnumb for all courses
crn_to_course = schedule_df.set_index('crn')['subj_crsnumb'].to_dict()

# Add main course subject to support_df
support_df['linked_subj_crsnumb'] = support_df['linked_crn'].map(crn_to_course)


In [90]:
#Create CRN → Campus Lookup
crn_to_campus_code = schedule_df.set_index('crn')['camp'].to_dict()
crn_to_campus = {crn: campus_lookup.get(code, code) for crn, code in crn_to_campus_code.items()}


In [91]:
# 📋 Prepare student roster lookup by CRN
# Creates: { (crn, student_id): row with name/email info }
crn_to_students = (
    roster_df.groupby('ssbsect_crn')
    .apply(lambda x: x.set_index('id')[['spriden_last_name', 'spriden_first_name', 'email']])
    .to_dict(orient='index')
)

<ipython-input-91-072222b830bc>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.set_index('id')[['spriden_last_name', 'spriden_first_name', 'email']])


In [92]:
def extract_info(student_ids, roster_df, crn):
    return [
        {
            'id': sid,
            'last': roster_df.loc[(roster_df['id'] == sid) & (roster_df['ssbsect_crn'] == crn), 'spriden_last_name'].values[0],
            'first': roster_df.loc[(roster_df['id'] == sid) & (roster_df['ssbsect_crn'] == crn), 'spriden_first_name'].values[0],
            'email': roster_df.loc[(roster_df['id'] == sid) & (roster_df['ssbsect_crn'] == crn), 'email'].values[0],
        }
        for sid in student_ids
        if ((roster_df['id'] == sid) & (roster_df['ssbsect_crn'] == crn)).any()
    ]


In [93]:
# 🔍 Compare student lists between main and support CRNs
seen_pairs = set()
results = []

for _, row in support_df.iterrows():
    main_crn = row['linked_crn']
    support_crn = row['crn']

    # Skip reversed duplicates
    pair_key = tuple(sorted([main_crn, support_crn]))
    if pair_key in seen_pairs:
        continue
    seen_pairs.add(pair_key)

    main_campus = crn_to_campus.get(main_crn, 'Unknown')
    support_campus = crn_to_campus.get(support_crn, 'Unknown')

    main_students = set(roster_df[roster_df['ssbsect_crn'] == main_crn]['id'])
    support_students = set(roster_df[roster_df['ssbsect_crn'] == support_crn]['id'])

    in_main_only = main_students - support_students
    in_support_only = support_students - main_students

    results.append({
      'main_crn': main_crn,
      'support_crn': support_crn,
      'linked_subj_crsnumb': row['linked_subj_crsnumb'],
      'support_subj_crsnumb': row['subj_crsnumb'],
      'main_campus': main_campus,
      'support_campus': support_campus,
      'in_main_only': extract_info(in_main_only, roster_df, main_crn),
      'in_support_only': extract_info(in_support_only, roster_df, support_crn)
})


In [94]:
# 🔢 Summary: Total CRN pairs checked
num_crn_pairs_checked = len(results)

# 🧑‍🎓 Summary: Total unique students checked across all CRNs
all_crns = set()
for entry in results:
    all_crns.update([entry['main_crn'], entry['support_crn']])

filtered_roster = roster_df[roster_df['ssbsect_crn'].isin(all_crns)]
unique_ids = filtered_roster['id'].nunique()

# 🖨️ Display summary
print("\n📊 Summary")
print(f"Total CRN pairs checked: {num_crn_pairs_checked}")
print(f"Total unique student IDs checked: {unique_ids}")



📊 Summary
Total CRN pairs checked: 38
Total unique student IDs checked: 650


In [95]:
# 🖨️ Print Only CRN Pairs with Mismatches
for entry in results:
    if not entry['in_main_only'] and not entry['in_support_only']:
        continue  # ✅ Skip if no mismatches

    print(
    f"\n\033[1m{entry['main_campus']}\033[0m\n"
    f"Main {entry['linked_subj_crsnumb']} CRN: {entry['main_crn']} | "
    f"Support {entry['support_subj_crsnumb']} CRN: {entry['support_crn']}\n"
)

    if entry['in_main_only']:
        print("➤ In Main Only:")
        for s in entry['in_main_only']:
            print(f"  - {s['id']} | {s['last']}, {s['first']} | {s['email']}")

    if entry['in_support_only']:
        print("➤ In Support Only:")
        for s in entry['in_support_only']:
            print(f"  - {s['id']} | {s['last']}, {s['first']} | {s['email']}")



Unknown
Main nan CRN: 54578 | Support MATH 0996 CRN: 54677

➤ In Support Only:
  - 002666893 | Clark, Mitchell | mclark16@student.gsu.edu
  - 002730080 | Pineda, Joshua | jpineda12@student.gsu.edu
  - 002864820 | Scott, Scott | sscott49@student.gsu.edu
  - 002845118 | Hicks, Laura | lhicks40@student.gsu.edu
  - 002307376 | Melendez, Linda | lmelendez12@student.gsu.edu
  - 002720903 | Frye, Keith | kfrye34@student.gsu.edu
  - 002793837 | Prince, Melissa | mprince15@student.gsu.edu
  - 002943267 | Mosley, Paul | pmosley25@student.gsu.edu
  - 002850896 | Robertson, Richard | rrobertson99@student.gsu.edu
  - 002284401 | Aguilar, Mary | maguilar94@student.gsu.edu
  - 002752940 | Silva, Brian | bsilva79@student.gsu.edu
  - 002745481 | Jimenez, Marilyn | mjimenez14@student.gsu.edu

Decatur
Main MATH 1111 CRN: 51608 | Support MATH 0999 CRN: 51606

➤ In Support Only:
  - 002954118 | Fox, James | jfox66@student.gsu.edu

Online
Main MATH 1001 CRN: 54751 | Support MATH 0997 CRN: 54750

➤ In Main 

# Create Excel Report

In [96]:
import pytz
from datetime import datetime

# Set timezone to US Eastern
eastern = pytz.timezone('US/Eastern')
now_et = datetime.now(eastern)

In [97]:
# Break it into parts to selectively format
month_day_year = now_et.strftime('%b_%d_%Y')  # e.g., Aug_08_2025 (keeps month capitalized)
hour_minute = now_et.strftime('%I_%M').lstrip('0')  # e.g., 2_30
am_pm = now_et.strftime('%p').lower()  # e.g., am or pm

# Combine to final timestamp
timestamp = f"{month_day_year}_{hour_minute}{am_pm}"
report_filename = f'Enrollment_Mismatch_Report_{timestamp}.xlsx'

In [98]:
# Ensure student IDs are strings with 9 digits (preserve 900 / 002 formats)
roster_df['id'] = roster_df['id'].astype(str).str.zfill(9)

# 📦 Flatten results into exportable rows
export_rows = []

for entry in results:
    if not entry['in_main_only'] and not entry['in_support_only']:
        continue

    for s in entry['in_main_only']:
        export_rows.append({
        'campus': entry['main_campus'],
        'main_crn': entry['main_crn'],
        'main_course': entry['linked_subj_crsnumb'],
        'support_crn': entry['support_crn'],
        'support_course': entry['support_subj_crsnumb'],
        'direction': 'In Main Only',
        'id': s['id'],
        'last': s['last'],
        'first': s['first'],
        'email': s['email'],
    })

    for s in entry['in_support_only']:
        export_rows.append({
        'campus': entry['main_campus'],
        'main_crn': entry['main_crn'],
        'main_course': entry['linked_subj_crsnumb'],
        'support_crn': entry['support_crn'],
        'support_course': entry['support_subj_crsnumb'],
        'direction': 'In Support Only',
        'id': s['id'],
        'last': s['last'],
        'first': s['first'],
        'email': s['email'],
    })

# Convert to DataFrame
mismatch_df = pd.DataFrame(export_rows)

# 💾 Export to Excel with ID column as text
with pd.ExcelWriter(report_filename, engine='xlsxwriter') as writer:
    mismatch_df.to_excel(writer, index=False, sheet_name='Mismatches')

    # Apply formatting to ensure IDs are treated as text
    workbook  = writer.book
    worksheet = writer.sheets['Mismatches']
    text_format = workbook.add_format({'num_format': '@'})

    # Find the column index of 'id' (in case column order changes)
    id_col_idx = mismatch_df.columns.get_loc('id')
    excel_col_letter = chr(ord('A') + id_col_idx)  # assumes < 26 columns
    worksheet.set_column(f'{excel_col_letter}:{excel_col_letter}', 15, text_format)


In [99]:
# 📤 Download from Colab
from google.colab import files
files.download(report_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>